In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import cv2

In [2]:
def convolve(image, K):
    
    #grabbing the spatial dimensions of image and kernel
    (iH, iW) = image.shape[:2]
    (kH, kW) = K.shape[:2]
    
    #allocating memory for output image, taking care to pad the boarders 
    #of the input image so that the spetial size(1.e, width and hight) are
    #not reduced
    pad = (kW - 1) // 2
    
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    
    output = np.zeros((iH, iW), dtype = 'float')
    
    #applying the convolution
    for y in np.arange(pad, iH+pad):
        for z in np.arange(pad, iW+pad):
            #roi is the small portion of image equal in dimensions to Kernal, that we will convolve with kernel
            roi = image[y-pad: y+pad+1, z-pad: z+pad+1]
            
            k = (roi * K).sum()     #actual convolution process
            
            output[y-pad, z-pad] = k
    output = rescale_intensity(output, in_range = (0, 255))
    output = (output * 255).astype('uint8')
            
    return output

In [3]:
#constructing average blurring kernels used to smooth an image
smallBlur = np.ones((7, 7), dtype = 'float')*(1.0 / (7*7))
largeBlur = np.ones((21, 21), dtype = 'float')*(1.0 / (21*21))

#constructing a sharpening filter
sharpen = np.array(([0, -1, 0], [-1, 5, -1], [0, -1, 0]), dtype = 'int')

#Laplacian Kernel for edge detection
laplacian = np.array(([0, 1, 0], [1, -4, 1], [0, 1, 0]), dtype = 'int')

#Sobel x-axis kernel
sobelX = np.array(([-1, 0, 1], [-2, 0, 2], [-1, 0, 1]), dtype = 'int')

#sobel y-axis kernel
sobelY = np.array(([-1, -2, 1], [0, 0, 0], [1, 2, 1]), dtype = 'int')

#construct an emboss kernel
emboss = np.array(([-2, -1, 0], [-1, 1, 1], [0, 1, 2]), dtype = 'int')

In [4]:
#constructing Kernel Bank
kernelBank = (
            ('small-blur', smallBlur),
            ('large-blur', largeBlur),
            ('sharpen', sharpen),
            ('laplacian', laplacian),
            ('sobel_x', sobelX),
            ('sobel_y', sobelY),
            ('emboss', emboss))

In [5]:
#loading the image and converting to grayscale
image = cv2.imread('jemma.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#loop over the kernels
for (kernelName, K) in kernelBank:
    
    #applying the kernel to the grayscale image using both our custom 'convolve'
    #function and OpenCV's 'filter2D' function
    
    print("[INFO] applying {} kernel".format(kernelName))
    convolveOutput = convolve(gray, K)
    opencvOutput = cv2.filter2D(gray, -1, K)
    
    #showing the output images
    cv2.imshow('Original', gray)
    cv2.imshow("{} - convolve".format(kernelName), convolveOutput)
    cv2.imshow("{} - opencv".format(kernelName), opencvOutput)
    cv2.waitKey(0)

[INFO] applying small-blur kernel
[INFO] applying large-blur kernel
[INFO] applying sharpen kernel
[INFO] applying laplacian kernel
[INFO] applying sobel_x kernel
[INFO] applying sobel_y kernel
[INFO] applying emboss kernel
